<a href="https://colab.research.google.com/github/vishallmaurya/sentiment_analyzer_backend/blob/main/predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!pip install FastAPI pymongo emoji load_dotenv

In [11]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
import pickle
import numpy as np
import emoji
from sentence_transformers import SentenceTransformer
from datetime import datetime
from pymongo import MongoClient
from text_preprocessing import cleaning_tweets

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [5]:
app = FastAPI()


In [ ]:
clf = pickle.load(open("/content/model.pkl", "rb"))
vectorizer = SentenceTransformer("all-MiniLM-L6-v2")

In [10]:
import os
from dotenv import load_dotenv

In [12]:
class TweetInput(BaseModel):
    tweet: str

def emoji_to_text(emoji_string):
    return ' '.join(emoji.demojize(e) for e in emoji_string) if emoji_string else ''

@app.post("/predict/")
async def predict_sentiment(tweet_input: TweetInput):
  try:
    tweet = tweet_input.tweet
    clean_text, extracted_emojis = cleaning_tweets(tweet)

    emoji_text = emoji_to_text(extracted_emojis)

    text_embedding = vectorizer.encode([clean_text], convert_to_numpy=True)
    embedding_dim = vectorizer.get_sentence_embedding_dimension()

    emoji_embedding = [
        vectorizer.encode(emoji_text, convert_to_numpy=True)
        if emoji_text.strip()
        else np.zeros(embedding_dim)
    ]

    text_features = np.hstack((text_embedding, emoji_embedding))

    sentiment_class = clf.predict(text_features)[0]

    return {"tweet": tweet, "predicted_class": sentiment_class}
  except Exception as e:
        raise HTTPException(status_code=500, detail=f"Error processing request: {str(e)}")